<a href="https://colab.research.google.com/github/gizdatalab/CPU_Tracs/blob/main/notebooks/IKI_TARGET_Setfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers==2.3.1
!pip install setfit==1.0.3
# !pip install datasets==2.3.0
# !pip install scikit-multilearn
# # !pip install optuna
# !pip install umap-learn hdbscan
# !pip install -U kaleido

In [ ]:
!pip install codecarbon

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datasets
import pandas as pd
import ast
import numpy as np
from typing import Dict, List, Optional, Set, Tuple, Union, Literal
from setfit import TrainingArguments
from setfit import Trainer
from transformers.trainer_callback import (
    CallbackHandler,
    DefaultFlowCallback,
    IntervalStrategy,
    PrinterCallback,
    ProgressCallback,
    TrainerCallback,
    TrainerControl,
    TrainerState,
)
from setfit import SetFitModel, TrainingArguments

from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers.losses.BatchHardTripletLoss import \
    BatchHardTripletLossDistanceFunction
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from setfit import SetFitModel
# to set the device type, if using GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import os
# import umap.umap_ as umap
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import sklearn.metrics as skm

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
path_to_folder = '/content/drive/MyDrive/Colab Notebooks/giz/IKI_Tracs/'

import json
import glob
import pandas as pd

files = glob.glob(path_to_folder + 'datasets/*.json' )
# read the json files
placeholder = []
for i,file in enumerate(files):
    placeholder.append(pd.read_json(file))
    print(len(placeholder[i]))

# drop out non-validated rows and some simple data manipulations
placeholder[0] = placeholder[0][placeholder[0].status == 'Validated']\
                                  .reset_index(drop=True)[['text','annotation']]
placeholder[0].replace({'ECONOMY-WIDE':'TARGET'}, inplace=True)
placeholder[1] = placeholder[1][placeholder[1].status == 'Validated']\
                                  .reset_index(drop=True)[['text','annotation']]

df = pd.concat(placeholder, ignore_index=True)
print(df.annotation.value_counts())

# separate out positive/negative examples
positive = df[df.annotation == 'TARGET'].drop_duplicates()
positive = positive.reset_index(drop=True)
negative = df[df.annotation == 'NEGATIVE'].drop_duplicates()
negative = negative.reset_index(drop=True)
# we will keep only limited negative examples in similar range of 'Target'
negative = negative.sample(n=100, random_state=42).reset_index(drop=True)
df = pd.concat([positive, negative], ignore_index=True)
print(df.annotation.value_counts())
# save the file as csv
df.to_csv('target_annotated.csv')

# Manual Inputs


In [ ]:
################ Define the important variables (Load and Process data) #######

# path to data folder
data_folder = '/content/'
file_name = 'target_annotated.csv'

# Map the columns which need to be processesed. i.e column which contains 'text' for
# prediciton and label column  for ground truth.
text_column = 'text'
label_column = 'annotation'

# train-test split
test_size=0.15


############# Define the Parameters for the Model to be used ##################

# Model to be used for Sentence Embedding
model_id = "GIZ/TAPP-multilabel-mpnet"

# USE DENSE ( Setfit Head) or NOT, If not Scikit Learn head will be used
use_differentiable_head=True
# normalize the embedding or not
normalize_embeddings: bool = True


############## Define Training Arguments   ########################

# batch sizes and epochs for the embedding and classifier training phases
# respectively, or set both if an integer is provided. Note that the batch
# size for the classifier is only used with a differentiable PyTorch head.
batch_size= (8,2)

# we keep the epochs=0 for classiifier stage as we will do the Classifier Trianing
# on our own using separate method, this is done to insure the we have control
# over the classifier trianing phase and look at metrics to guage if
# overfitting or underfititng
num_epochs = (1,0)


# we need it for our own custom Classifier training
classifier_num_epochs = 10


# for multi-class and multilabel dataset we will usually
# have positive class in minority there fore maybe a good
# option to try, rather than model getting focused too much
# on negative class.
# [''oversampling'.'undersampling, 'unique'], Default = 'oversampling'
sampling_strategy = 'undersampling'

# Set the learning rate for the SentenceTransformer body for the embedding and
# classifier training phases respectively, or set both if a float is provided.
# Note that the body learning rate for the classifier is only used with a
# differentiable PyTorch head and if end_to_end=True.
body_learning_rate = (0.00002, 0.00001)
head_learning_rate = 0.01


loss = losses.CosineSimilarityLoss

# to be used only for triplet loss categories
# distance_metric = losses.BatchHardTripletLossDistanceFunction.cosine_distance,

#  log and evlautions strategy and steps
# Strategy possible vlaues: ["steps","epochs","no"]
logging_strategy = "steps"
evaluation_strategy = "steps"
logging_steps = 500
eval_steps=500

# # to be used for triplet loss category
# margin = 0.25

## to be used with differential head
end_to_end = False
# if use_differentiable_head == True:
#     end_to_end = True
# else: end_to_end = False

warmup_proportion = 0.01

## to be used for triplet loss only
# samples_per_label = 2

# output dir, if you push the mdoel to hub then this will be name of model
output_dir = "setfit_iki_target"


# method to use for embedding dimentionality reduction
method = 'tnse'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Automated

## Dataset

In [ ]:
import pandas as pd
import ast
import numpy as np

In [ ]:
df = pd.read_csv(data_folder + file_name, index_col=0)

# get the categories or labels information
label_names = list(df[label_column].unique())
label_names.sort()
num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print("Label Mapping",'\n')
for key,val in label2id.items(): print(key,":",val)

# create one-hot-encoded labels
df['binary'+label_column] = df[label_column].apply(lambda x: label2id[x])

Label Mapping 

NEGATIVE : 0
TARGET : 1


/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [ ]:
from sklearn.model_selection import train_test_split
df = df.sample(frac=1.0, random_state=42)
df_train, df_test = train_test_split(df, test_size=test_size, random_state= 42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


print('\n','Classes Representation after stratification', "Training data:",len(df_train))
print(df_train['binary'+label_column].value_counts())

print('\n','Classes Representation after stratification', "Test data:",len(df_test))
print(df_test['binary'+label_column].value_counts())

df_train = df_train[['text','binaryannotation']]
df_train.rename(columns= {'binaryannotation':'label'}, inplace=True)
df_test = df_test[['text','binaryannotation']]
df_test.rename(columns= {'binaryannotation':'label'}, inplace=True)
df_train['categorical_label'] = df_train['label']
df_test['categorical_label'] = df_test['label']


 Classes Representation after stratification Training data: 214
0    172
1     42
Name: binaryannotation, dtype: int64

 Classes Representation after stratification Test data: 38
0    28
1    10
Name: binaryannotation, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train_ds = datasets.Dataset.from_pandas(df_train)
test_ds = datasets.Dataset.from_pandas(df_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Model

In [ ]:
if use_differentiable_head == True:
    model = SetFitModel.from_pretrained(
                      model_id,
                      labels = label_names,
                      device= device,
                      normalize_embeddings = normalize_embeddings,
                      use_differentiable_head=use_differentiable_head,
                      )
else:
    model = SetFitModel.from_pretrained(
              model_id,
              device= device,
              normalize_embeddings = normalize_embeddings,
              labels = label_names)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at GIZ/TAPP-multilabel-mpnet and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and infer

## Metrics

In [ ]:
def compute_metrics(y_pred, y_test):
    report = skm.classification_report(y_test, y_pred, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    return {"Precision_macro": df_report.loc['macro avg']['precision'],
            "Precision_weighted": df_report.loc['weighted avg']['precision'],
            "Recall_macro": df_report.loc['macro avg']['recall'],
            "Recall_weighted": df_report.loc['weighted avg']['recall'],
            "F1-Score": df_report.loc['macro avg']['f1-score'],
            "Accuracy": skm.accuracy_score(y_test, y_pred)}

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir = output_dir,
    batch_size = batch_size,
    num_epochs = num_epochs,
    sampling_strategy = sampling_strategy,
    body_learning_rate = body_learning_rate,
    head_learning_rate = head_learning_rate,
    loss = loss,
    end_to_end = end_to_end,
    warmup_proportion = warmup_proportion,
    logging_strategy=logging_strategy,
    logging_steps = logging_steps,
    eval_steps=eval_steps,
    evaluation_strategy = evaluation_strategy,
)

## Callbacks

In [ ]:
def umap_embed(embeddings, n_neighbors = 15, n_dim = 2, eval_embeddings = None):
    """
    Method to use UMAP to compress the embeddings into n_dim dimensions

    """
    umap_model = (umap.UMAP(n_neighbors=n_neighbors,
                                n_components=n_dim,
                                metric='cosine',
                                random_state=42))
    train_embed = umap_model.fit_transform(embeddings.cpu())

    train_umap = pd.DataFrame()
    for i in range(n_dim):
        train_umap[f'dim_{i+1}'] = train_embed[:,i]
    if eval_embeddings is not None:
        eval_embed = umap_model.transform(eval_embeddings.cpu())
        eval_umap = pd.DataFrame()
        for i in range(n_dim):
            eval_umap[f'dim_{i+1}'] = eval_embed[:,i]

        return train_umap, eval_umap
    return train_umap, None


def tnse_embed(train_embeddings,n_neighbors = 15, n_dim = 2):
    """
    Method to use TSNE to compress the embeddings into n_dim dimensions
    ####### NOTE Why not FIT_TRansform  ################

    """

    train_embed = TSNE(n_components=n_dim, perplexity=n_neighbors,
                   random_state=42).fit_transform(train_embeddings.cpu())
    train_tnse = pd.DataFrame()
    for i in range(n_dim):
        train_tnse[f'dim_{i+1}'] = train_embed[:,i]

    return train_tnse


def plot_embeddings_plotly(model:SetFitModel,train_data, n_neighbors,
                           method:Literal["tnse", "umap"] = "tnse",
                           eval_data=None, n_dim = 3):
    """
    given the text data, it uses the model to find the embeddings of text data,
    then compresses them to n_dim and makes the plot using plotly. You can pass
    just training data or train and val data for comparison

    """
    train_embeddings = model.encode(train_data["text"])
    if method  == 'tnse':
        scatter_df = tnse_embed(train_embeddings, n_neighbors,n_dim)
        scatter_df['labels'] = list(train_data['label_text'])
        scatter_df['type'] = "Training Embeddings"
        if eval_data is not None:
            eval_embeddings = model.encode(eval_data["text"])
            eval_scatter_df = tnse_embed(eval_embeddings,n_neighbors,n_dim)
            eval_scatter_df['labels'] = list(eval_data['label_text'])
            eval_scatter_df['type'] = "Validation Embeddings"
            scatter_df = pd.concat([scatter_df,eval_scatter_df],ignore_index=True)

    if method  == 'umap':
        if eval_data is not None:
            eval_embeddings = model.encode(eval_data["text"])
            scatter_df, eval_scatter_df = umap_embed(train_embeddings, n_neighbors,n_dim,eval_embeddings)
            scatter_df['labels'] = list(train_data['label_text'])
            scatter_df['type'] = "Training Embeddings"
            eval_scatter_df['labels'] = list(eval_data['label_text'])
            eval_scatter_df['type'] = "Validation Embeddings"
            scatter_df = pd.concat([scatter_df,eval_scatter_df],ignore_index=True)
        else:
            scatter_df,_ = umap_embed(train_embeddings, n_neighbors,n_dim,eval_embeddings)
            scatter_df['labels'] = list(train_data['label_text'])
            scatter_df['type'] = "Training Embeddings"

    if n_dim == 3:
        fig = px.scatter_3d(
        scatter_df, x='dim_1', y='dim_2', z='dim_3',facet_col="type",
        #color_discrete_sequence=['#bbbabb'],
        color='labels',
        opacity = .5)
        fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
        fig.update_traces(marker_size=3)
    else:
        fig = px.scatter(scatter_df, x='dim_1', y='dim_2', color = 'labels', facet_col="type")

    return fig


def plot_embeddings(model:SetFitModel,train_data, n_neighbors,
                           method:Literal["tnse", "umap"] = "tnse",
                           eval_data=None, n_dim = 2):
    """
    given the text data, it uses the model to find the embeddings of text data,
    then compresses them to n_dim and makes the plot using simple matplotlib.
    You can pass just training data or train and val data for comparison

    """
    if method not in ["tnse","umap"]:
      print("method value is wrong, excepted values are ['tnse','umap']")
      return None

    train_embeddings = model.encode(train_data["text"])
    if method  == 'tnse':
            scatter_df = tnse_embed(train_embeddings, n_neighbors,n_dim)
            scatter_df['labels'] = list(train_data['categorical_label'])
            if eval_data is not None:
                eval_embeddings = model.encode(eval_data["text"])
                eval_scatter_df = tnse_embed(eval_embeddings,n_neighbors,n_dim)
                eval_scatter_df['labels'] = list(eval_data['categorical_label'])

    if method  == 'umap':
        if eval_data is not None:
            eval_embeddings = model.encode(eval_data["text"])
            scatter_df, eval_scatter_df = umap_embed(train_embeddings, n_neighbors,n_dim,eval_embeddings)
            scatter_df['labels'] = list(train_data['categorical_label'])
            eval_scatter_df['labels'] = list(eval_data['categorical_label'])
        else:
            scatter_df,_ = umap_embed(train_embeddings, n_neighbors,n_dim,eval_embeddings)
            scatter_df['labels'] = list(train_data['categorical_label'])

    fig, (train_ax, eval_ax) = plt.subplots(ncols=1,nrows=2, figsize = (10,20))

    train_scatter  = train_ax.scatter(x= scatter_df['dim_1'],
                                      y= scatter_df['dim_2'],
                                      c=scatter_df["labels"],alpha = 0.4)
                                          # label=list(df_train_explode["categorical_label"]))
    train_ax.legend(handles=train_scatter.legend_elements(num=len(label_names))[0],
                    labels=label_names) #,loc='upper center', bbox_to_anchor=(0.5, -0.05),
                #    fancybox=True, ncol=5)
    train_ax.set_title("Training embeddings")


    eval_scatter  = eval_ax.scatter(x= eval_scatter_df['dim_1'],
                                      y= eval_scatter_df['dim_2'],
                                      c=eval_scatter_df["labels"], alpha =0.4)
    eval_ax.legend(handles=eval_scatter.legend_elements(num=len(label_names))[0],
                        labels=label_names) #,loc='upper center', bbox_to_anchor=(0.5, -0.05),
                     #   fancybox=True, ncol=5)
    eval_ax.set_title("Evaluation embeddings")
    return fig

In [ ]:
class MetricsCallback(TrainerCallback):
    """Simple Metrics call back."""
    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, model: SetFitModel, metrics, **kwargs):
        val_x = df_test["text"]
        val_y = df_test["label"]

        train_x = df_train["text"]
        train_y = df_train["label"]

        y_pred_train = model.predict(train_x, use_labels=False)
        y_pred_val = model.predict(val_x, use_labels=False)

        if isinstance(y_pred_train, torch.Tensor):
            y_pred_train = y_pred_train.cpu()

        if isinstance(y_pred_val, torch.Tensor):
            y_pred_val= y_pred_val.cpu()

        train_m = compute_metrics(y_pred_train, train_y)
        val_m = compute_metrics(y_pred_val, val_y)
        print(train_m)
        print(val_m)

class EmbeddingPlotCallbackPlotly(TrainerCallback):
    """Simple embedding plotting callback that plots the embeddings of training and evaluation datasets throughout training
    using plotly ."""
    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, model: SetFitModel, **kwargs):
        directory = 'embeddings_plots'
        if not os.path.exists(directory):
            os.makedirs(directory)

        fig = plot_embeddings_plotly(model,df_train, num_labels,method,df_test,n_dim=2)
        fig.show()
        fig.write_image(f"{directory}/step_{state.global_step}.png")

class EmbeddingPlotCallback(TrainerCallback):
    """Simple embedding plotting callback that plots the embeddings of the training and evaluation datasets throughout training."""
    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, model: SetFitModel, **kwargs):
        directory = 'embeddings_plots'

        if not os.path.exists(directory):
            os.makedirs(directory)

        fig  = plot_embeddings(model,df_train, num_labels,method,df_test,n_dim=2)
        fig.suptitle(f"tSNE of training and evaluation embeddings at step {state.global_step} of {state.max_steps}.")
        fig.savefig(f"{directory}/step_{state.global_step}.png")
        plt.show()

## Custom Trainer

In [ ]:
from tqdm.auto import tqdm, trange
from typing import Dict, List, Optional, Set, Tuple, Union, Literal
import joblib
import numpy as np
import requests
import torch
from huggingface_hub import PyTorchModelHubMixin, hf_hub_download
from huggingface_hub.utils import validate_hf_hub_args
from sentence_transformers import SentenceTransformer, models
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain, MultiOutputClassifier
from torch import nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from transformers.utils import copy_func

metrics = {"training":[],"validation":[]}
class MultilabelTrainer(Trainer):
        def on_evaluate_(self):
            val_x = list(df_test["text"])
            val_y = list(df_test["label"])

            train_x = list(df_train["text"])
            train_y = list(df_train["label"])

            y_pred_train = self.model.predict(train_x, use_labels=False)
            y_pred_val = self.model.predict(val_x, use_labels=False)

            if isinstance(y_pred_train, torch.Tensor):
                y_pred_train = y_pred_train.cpu()

            if isinstance(y_pred_val, torch.Tensor):
                y_pred_val= y_pred_val.cpu()

            train_m = compute_metrics(y_pred_train, train_y)
            val_m = compute_metrics(y_pred_val, val_y)
            print("training metrics",'\n',train_m)
            print("eval metrics",'\n',val_m)

            metrics['training'].append(train_m)
            metrics['validation'].append(val_m)


        def train_classifier_new(
        self, x_train: List[str], y_train: Union[List[int], List[List[int]]],
        x_eval: List[str], y_eval: Union[List[int], List[List[int]]],
        args: Optional[TrainingArguments] = None, classifier_num_epochs =classifier_num_epochs
    ) -> None:
            """
            Method to perform the classifier phase: fitting a classifier head.

            Args:
                x_train (`List[str]`): A list of training sentences.
                y_train (`Union[List[int], List[List[int]]]`): A list of labels corresponding to the training sentences.
                args (`TrainingArguments`, *optional*):
                    Temporarily change the training arguments for this training call.
            """
            args = args or self.args or TrainingArguments()

            num_epochs=classifier_num_epochs
            batch_size=args.classifier_batch_size
            body_learning_rate=args.body_classifier_learning_rate
            head_learning_rate=args.head_learning_rate
            l2_weight=args.l2_weight
            max_length=args.max_length
            show_progress_bar=args.show_progress_bar
            end_to_end=args.end_to_end


            if self.model.has_differentiable_head:  # train with pyTorch
              self.model.model_body.train()
              self.model.model_head.train()
              if not end_to_end:
                  self.freeze("body")

              dataloader = self.model._prepare_dataloader(x_train, y_train, batch_size, max_length)
              criterion = self.model.model_head.get_loss_fn()
              optimizer = self.model._prepare_optimizer(head_learning_rate, body_learning_rate, l2_weight)
              scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
              for epoch_idx in trange(num_epochs, desc="Epoch", disable=not show_progress_bar):
                  print(f"Epoch Value:{epoch_idx}")
                  for batch in tqdm(dataloader, desc="Iteration", disable=not show_progress_bar, leave=False):
                      features, labels = batch
                      optimizer.zero_grad()

                      # to model's device
                      features = {k: v.to(self.model.device) for k, v in features.items()}
                      labels = labels.to(self.model.device)


                      outputs = self.model.model_body(features)
                      if self.model.normalize_embeddings:
                          outputs["sentence_embedding"] = nn.functional.normalize(
                              outputs["sentence_embedding"], p=2, dim=1
                          )
                      outputs = self.model.model_head(outputs)
                      logits = outputs["logits"]

                      loss: torch.Tensor = criterion(logits, labels)
                      loss.backward()
                      optimizer.step()

                  scheduler.step()
                  self.on_evaluate_()

              if not end_to_end:
                  self.model.unfreeze("body")
            else:  # train with sklearn
                embeddings = self.model.model_body.encode(x_train, normalize_embeddings=self.normalize_embeddings)
                self.model.model_head.fit(embeddings, y_train)

# Training

In [ ]:
# use appropriate callbacks that you want to include
trainer = MultilabelTrainer(model = model,
          args = args,
          train_dataset=train_ds,
          eval_dataset=test_ds,
          metric = compute_metrics,
          column_mapping={"text": "text", "label": "label"},
          callbacks=[EmbeddingPlotCallback()])

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/214 [00:00<?, ? examples/s]

## Embedding Training

In [ ]:
# if emebddings visulization are selected in callbacks then you can find the plots in
# folder "embeddings_plots" in root directory.
trainer.train()

In [ ]:
report = pd.DataFrame(trainer.model.model_card_data['eval_lines_list'])
report

## Classifier training

In [ ]:
import warnings
warnings.simplefilter('ignore')
trainer.train_classifier_new(list(df_train['text']),list(df_train['label']),list(df_test['text']),list(df_test['label']))

In [ ]:
tracker.stop()

0.012375304838414763

In [ ]:
metrics_training = pd.DataFrame(metrics['training'])
metrics_training = metrics_training.add_prefix('training_')
metrics_val = pd.DataFrame(metrics['validation'])
metrics_val  = metrics_val.add_prefix('val_')
metrics_df = pd.concat([metrics_training,metrics_val],axis=1)
display(metrics_df)
cols = list(metrics_df.columns)
cols_filtered = list(filter(lambda x: 'f1' in x, cols))
display(metrics_df[cols_filtered])

In [ ]:
trainer.push_to_hub("ppsingh/iki_target_setfit")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/7.70k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/iki_target_setfit/commit/fdaa60f1df10bb7b4299360778f418a5cbbcf341', commit_message='Add SetFit model', commit_description='', oid='fdaa60f1df10bb7b4299360778f418a5cbbcf341', pr_url=None, pr_revision=None, pr_num=None)

# Predict and Evaluate

## Trainer

In [ ]:
model = SetFitModel.from_pretrained("ppsingh/iki_target_setfit")

config.json:   0%|          | 0.00/873 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/7.70k [00:00<?, ?B/s]

In [ ]:
import json
import glob
import pandas as pd
files = glob.glob(path_to_folder + 'datasets/*.json' )
placeholder = []
for i,file in enumerate(files):
    placeholder.append(pd.read_json(file))
    print(len(placeholder[i]))

placeholder[0] = placeholder[0][placeholder[0].status == 'Validated']\
                                  .reset_index(drop=True)[['text','annotation']]
placeholder[0].replace({'ECONOMY-WIDE':'TARGET'}, inplace=True)
placeholder[1] = placeholder[1][placeholder[1].status == 'Validated']\
                                  .reset_index(drop=True)[['text','annotation']]

df = pd.concat(placeholder, ignore_index=True)

436
65


In [ ]:
import sklearn.metrics as skm
# Run inference
y_pred = model(list(df.text))
y_pred = [0 if x == 'NEGATIVE' else 1 for x in y_pred]
y_true = list(df.annotation)
y_true = [0 if x == 'NEGATIVE' else 1 for x in y_true]
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

NEGATIVE
[[ 50   2]
 [ 59 380]] 

TARGET
[[380  59]
 [  2  50]] 

